In [ ]:
library(rstan)
library(tidyverse)
library(ggmcmc)
library(tidybayes)
library(lubridate)

# MCMC sampling

## load your data

In [91]:
d <- read_csv("time_series.csv") %>%
  arrange(year, month)

data <- list()
data$T <- nrow(d)
data$Y <- d$pay / max(d$pay)
data$T_pred <- 12

Parsed with column specification:
cols(
  year = col_integer(),
  month = col_integer(),
  pay = col_integer()
)


## small sampling

In [ ]:
fit <- stan(file = 'seasonality.stan', data = data, seed = 0,
            iter = 700, warmup = 200, chains = 3, control = list(adapt_delta=0.99))

## finally, large sampling

In [95]:
fit <- stan(file = 'seasonality.stan', data = data, seed = 0,
    iter = 3000, warmup = 500, chains = 4, control = list(adapt_delta=0.99))


SAMPLING FOR MODEL 'seasonality' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 5.2e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.52 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 3000 [  0%]  (Warmup)
Chain 1: Iteration:  300 / 3000 [ 10%]  (Warmup)
Chain 1: Iteration:  501 / 3000 [ 16%]  (Sampling)
Chain 1: Iteration:  800 / 3000 [ 26%]  (Sampling)
Chain 1: Iteration: 1100 / 3000 [ 36%]  (Sampling)
Chain 1: Iteration: 1400 / 3000 [ 46%]  (Sampling)
Chain 1: Iteration: 1700 / 3000 [ 56%]  (Sampling)
Chain 1: Iteration: 2000 / 3000 [ 66%]  (Sampling)
Chain 1: Iteration: 2300 / 3000 [ 76%]  (Sampling)
Chain 1: Iteration: 2600 / 3000 [ 86%]  (Sampling)
Chain 1: Iteration: 2900 / 3000 [ 96%]  (Sampling)
Chain 1: Iteration: 3000 / 3000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 7.81332 seconds (Warm-up)
Chain 1:                55.7256 seconds (Sampling)
Chain 1:       

Warning message:
“There were 6 divergent transitions after warmup. Increasing adapt_delta above 0.99 may help. See
http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”Warning message:
“There were 9558 transitions after warmup that exceeded the maximum treedepth. Increase max_treedepth above 10. See
http://mc-stan.org/misc/warnings.html#maximum-treedepth-exceeded”Warning message:
“There were 4 chains where the estimated Bayesian Fraction of Missing Information was low. See
http://mc-stan.org/misc/warnings.html#bfmi-low”Warning message:
“Examine the pairs() plot to diagnose sampling problems
”

# convergence diagnosis
## trace plot

In [96]:
ggmcmc(ggs(fit, inc_warmup = T, stan_include_auxiliar = T), plot = "traceplot",
       file = "./trace.pdf")

Plotting traceplots
Time taken to generate the report: 230 seconds.


## list vars arranged in Rhat order

In [106]:
broom::tidy(fit, rhat=T) %>%
    filter(str_detect(term, "_all\\[[0-9]+\\]$|_pred\\[[0-9]+\\]$") == F) %>%
    filter(str_detect(term, "^former_month") == F) %>%
    select(term, rhat) %>%
    arrange(-rhat) %>% 
    head()

term,rhat
s_month,1.040647
month[25],1.039185
month[13],1.037469
month[37],1.036549
month[49],1.033201
month[1],1.032677


# visualization
## tidy pre-proccessing

In [3]:
ms <- fit %>%
    spread_draws(mu_all[t], month_all[t]) %>%
    median_qi(.width=0.8, .simple_names=F) %>%
    select(-.width, -.point, -.interval) %>%
    left_join(
        spread_draws(fit, y_pred[t]) %>%
            median_qi(.width=0.8, .simple_names=F) %>%
            mutate(t = t+nrow(d)) %>%
            select(-.width, -.point, -.interval),
        by = 't'
    )

## ggplot

In [ ]:
options(repr.plot.width=8, repr.plot.height=4)
ms %>%
    ggplot(
        aes(date)
    ) +
    geom_line(aes(y = y_pred)) +
    geom_ribbon(aes(ymax = y_pred.upper, ymin = y_pred.lower), alpha=1/3) +
    geom_line(aes(y = mu_all), color="red") +
    geom_ribbon(aes(ymax = mu_all.upper, ymin = mu_all.lower), alpha=1/5, fill="red") +
    scale_y_continuous(labels = scales::comma) +
    scale_x_date(date_breaks = "1 month", date_labels = "%Y-%m") +
    theme(axis.text.x = element_text(angle = 90, hjust = 1))